<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
🗺️ תהליך הסבת שכבת קדסטר מבוסס קואורדינטות לבנק"ל מודרני 🗺️ 

In [ ]:
print('X. Coordinates Based Cadastre Borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import datetime as dt
import pandas as pd
import geopandas as gpd
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('  X.a Reading inputs')

In [ ]:
RawData_Folder = r'\\mapi_shares\MNCDB\צוות מידע\RawData/'

today = dt.date.today()
product_name = str(today.year) + str(today.month) + str(today.day) + '_ncdb.gdb'
product_path = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים/' + product_name
del [today, product_name]

<h1 style="text-align:right;font-size:100%">
טבלת מספרי גושים של קמ"ק

In [ ]:
CBC_list = pd.read_excel(RawData_Folder + 'Local\CoordBasedCadstreBlocksList.xlsx', usecols = ['BlockNumber','SubBlockNumber'])

<h1 style="text-align:right;font-size:100%">
שכבת גושי קדסטר מודרנית

In [ ]:
Blocks_cols = ['BlockNumber', 'SubBlockNumber', 'BlockUniqueID']

Blocks = gpd.read_file(product_path,
                       layer = 'Blocks',
                       include_fields = Blocks_cols,
                       ignore_geometry = False)

del[Blocks_cols, RawData_Folder, product_path]

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('  X.b Cleaning data')

In [ ]:
print('  X.c Data conversion')

<h1 style="text-align:right;font-size:100%">
מיזוג מידע

In [ ]:
merge = Blocks.merge(CBC_list, on = ['BlockNumber','SubBlockNumber'], how = 'right')
merge = merge.drop_duplicates(keep='first')
merge = merge[~merge['geometry'].isna()]

del[CBC_list, Blocks]

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('  X.d Defining new Layer')

In [ ]:
new_columns = np.dtype([
                        ("CPBUniqueID", int),
                        #("BlockUniqueID", int), יאוחד בהמשך התהליך
                        ("ProcessType", int),
                        ("MapYear", int),
                        ("ProcessName", str),
                        ("GeodeticNetwork", int),
                        ("Status", int),
                        ("PlanName", str),
                        ("SurveyorLicenseID", int),
                        ("DataSource", int),
                        ("geometry", gpd.array.GeometryDtype)
                        ])

CBC_Blocks = gpd.GeoDataFrame(data     = np.empty(0, dtype = new_columns),
                              geometry = 'geometry',
                              crs      = 2039)

<h1 style="text-align:right;font-size:100%">
איכלוס שדות

In [ ]:
CBC_Blocks['BlockUniqueID'] = merge['BlockUniqueID'].astype('Int64')

CBC_Blocks['ProcessType'] = 5

CBC_Blocks['MapYear'] = 2013

CBC_Blocks['ProcessName'] = 'CBC' + '-' + merge['BlockNumber'].astype(str) + '/' + merge['SubBlockNumber'].astype(str)

CBC_Blocks['GeodeticNetwork'] = 3

CBC_Blocks['Status'] = 23 #מאושר

CBC_Blocks['PlanName'] = None

CBC_Blocks['SurveyorLicenseID'] = None

CBC_Blocks['DataSource'] = 5 #CAD

CBC_Blocks['geometry'] = merge['geometry']

CBC_Blocks['CPBUniqueID'] = range(5000000, 5000000+len(CBC_Blocks))

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
print('  X.e Exporting results to product folder')

In [ ]:
#csv_dir  = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\csv\CoordinatesBasedCadastreBorders.csv'
json_dir =  r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\json\CoordinatesBasedCadastreBorders.geojson'

#CBC_Blocks.drop(columns='geometry').to_csv(csv_dir ,encoding='cp1255', index=False)

CBC_Blocks.to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([r"M:\ncdbScripts\scripts\arcpyPro\CoordinatesBasedCadastreBorders - Save as Feature Class.bat"]);